# Pretraining `num_rows` vs transfered performance
Pretrain fastsim weights for 1M, 2M, 4M, 8M, ... rows

Then transfer and do fixed fullsim transfer training

In [1]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from tqdm import tqdm
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import initializers

import wandb
from wandb.keras import WandbCallback

# https://gitlab.cern.ch/atlas/ATLAS-top-tagging-open-data/-/blob/master/preprocessing.py
import preprocessing

np.random.seed(8)
tf.random.set_seed(8)

physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-05-19 02:46:57.046284: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-05-19 02:46:57.078873: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:41:00.0 name: NVIDIA A40 computeCapability: 8.6
coreClock: 1.74GHz coreCount: 84 deviceMemorySize: 44.56GiB deviceMemoryBandwidth: 648.29GiB/s
2023-05-19 02:46:57.079640: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-05-19 02:46:57.083264: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-05-19 02:46:57.086610: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2023-05-19 02:46:57.087622: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2023-0

In [2]:
train_file_names = os.listdir("/global/ml4hep/spss/mfong/transfer_learning/delphes_train")
# for i in range(7, 15):
for i in range(1, 15):
    train_file_names.remove(f"train_{i}.h5")
train_file_names
# f = h5py.File('/clusterfs/ml4hep/mfong/transfer_learning/delphes_train.h5', 'r')
# f2 = h5py.File('/clusterfs/ml4hep/mfong/transfer_learning/delphes_test.h5', 'r')

['train_0.h5']

In [3]:
num_samples_per_file = []
for train_file_name in train_file_names:
    f = h5py.File("/global/ml4hep/spss/mfong/transfer_learning/delphes_train/" + train_file_name, 'r')
    num_samples_per_file.append(f["fjet_clus_eta"].shape[0])
num_samples = sum(num_samples_per_file)
feature_keys = ['fjet_clus_eta', 'fjet_clus_phi', 'fjet_clus_pt', 'fjet_clus_E']
num_features = 7*200
# for k in feature_keys:
#     num_features += f[k].shape[1]
x = np.empty((num_samples, num_features))

In [4]:
x.shape

(5000000, 1400)

In [5]:
current_row = 0
for train_file_name, current_num_samples in tqdm(zip(train_file_names, num_samples_per_file)):
    f = h5py.File("/global/ml4hep/spss/mfong/transfer_learning/delphes_train/" + train_file_name, 'r')
    
    # preprocess
    data_dict = {k:v for k, v in f.items() if k in feature_keys}
    x[current_row:current_row+current_num_samples] = preprocessing.constituent(data_dict, 200).reshape(f["fjet_clus_eta"].shape[0], x.shape[1])       # TODO need to put these into preallocated x array
    # x = preprocessing.high_level(x)
    
    # x[current_row:current_row+current_num_samples] = np.concatenate([f[k] for k in feature_keys], axis=1)
    current_row += current_num_samples

0it [00:00, ?it/s]/global/home/users/mfong/git/transfer-learning/preprocessing.py:90: RuntimeWarning: divide by zero encountered in log
  log_pt = np.log(pt)
/global/home/users/mfong/git/transfer-learning/preprocessing.py:91: RuntimeWarning: divide by zero encountered in log
  log_energy = np.log(energy)
/global/home/users/mfong/git/transfer-learning/preprocessing.py:98: RuntimeWarning: divide by zero encountered in log
  lognorm_pt = np.log(pt / sum_pt[:,np.newaxis])
/global/home/users/mfong/git/transfer-learning/preprocessing.py:99: RuntimeWarning: divide by zero encountered in log
  lognorm_energy = np.log(energy / sum_energy[:,np.newaxis])
1it [04:42, 282.67s/it]


In [6]:
feature_keys = ['fjet_clus_eta', 'fjet_clus_phi', 'fjet_clus_pt', 'fjet_clus_E']
for k in f.keys():
    print(k, f[k].shape)

fjet_clus_E (5000000, 200)
fjet_clus_eta (5000000, 200)
fjet_clus_phi (5000000, 200)
fjet_clus_pt (5000000, 200)
fjet_eta (5000000,)
fjet_m (5000000,)
fjet_phi (5000000,)
fjet_pt (5000000,)
labels (5000000,)
training_weights (5000000,)


In [7]:
y = np.empty((num_samples))
current_row = 0
for train_file_name, current_num_samples in tqdm(zip(train_file_names, num_samples_per_file)):
    f = h5py.File("/global/ml4hep/spss/mfong/transfer_learning/delphes_train/" + train_file_name, 'r')
    y[current_row:current_row+current_num_samples] = f["labels"][:]
    current_row += current_num_samples

1it [00:00,  1.92it/s]


In [8]:
x.shape

(5000000, 1400)

In [9]:
y.shape

(5000000,)

In [10]:
num_samples = len(y)
num_train_samples = int(0.8 * num_samples)
# num_train_samples = num_samples - 2000000       # save 2M rows for test data
x_train = x[:num_train_samples]
y_train = y[:num_train_samples]

x_test = x[num_train_samples:]
y_test = y[num_train_samples:]

In [11]:
x_train.shape

(4000000, 1400)

In [12]:
x_test.shape

(1000000, 1400)

In [13]:
# scaler = StandardScaler()
# scaler.fit(x_train[:2000000])   # only use first 2M otherwise takes too long

# x_train = scaler.transform(x_train, copy=False)
# x_test = scaler.transform(x_test, copy=False)

In [14]:
os.environ["WANDB_NOTEBOOK_NAME"] = "pretrain_MLP.ipynb"

In [ ]:
# NUM_PRETRAIN_ROWS_LIST = [1000000, 2000000, 4000000, 8000000, 16000000, 32000000]
NUM_PRETRAIN_ROWS_LIST = [100000, 200000, 400000, 800000, 1000000]

# config = wandb.config
# config.batch_size = 256
config = {
    "batch_size": 256,
    "epochs": 400,
}
for num_pretrain_rows in NUM_PRETRAIN_ROWS_LIST:
    # config.num_pretrain_rows = num_pretrain_rows
    config["num_pretrain_rows"] = num_pretrain_rows
    run = wandb.init(project="pretrain_MLP", name=f"preprocess_fastsim_MLP_{int(num_pretrain_rows / 1000)}K_rows", config=config, reinit=True)
    
    model = Sequential()
    model.add(Dense(64, input_shape=(x_train.shape[1],), activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(
        x_train[:config["num_pretrain_rows"]],
        y_train[:config["num_pretrain_rows"]],
        epochs=config["epochs"],
        batch_size=config["batch_size"],
        shuffle=True,
        validation_data=(x_test, y_test),
        callbacks=[wandb.keras.WandbCallback()]
    )
    
    plt.figure()
    plt.plot(history.history["accuracy"], label="acc")
    plt.plot(history.history["val_accuracy"], label="val_acc")
    plt.title(f"Preprocessed Fastsim MLP ({int(num_pretrain_rows / 1000)}K Rows)")
    plt.ylabel("Accuracy")
    plt.xlabel("Epoch")
    plt.legend()
    plt.savefig(f"output/preprocess_fastsim_MLP_{int(num_pretrain_rows / 1000)}K_rows_acc.png")
    
    plt.figure()
    plt.plot(history.history["loss"], label="loss")
    plt.plot(history.history["val_loss"], label="val_loss")
    plt.title(f"Preprocessed Fastsim MLP ({int(num_pretrain_rows / 1000)}K Rows)")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.legend()
    plt.savefig(f"output/preprocess_fastsim_MLP_{int(num_pretrain_rows / 1000)}K_rows_loss.png")
    
    
    
    model.save_weights(f"models/preprocess_fastsim_MLP_{int(num_pretrain_rows / 1000)}K_rows.h5")
    wandb.finish()

wandb: Currently logged in as: mingfong. Use `wandb login --relogin` to force relogin


2023-05-19 02:52:04.621494: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2023-05-19 02:52:04.637520: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2994330000 Hz
2023-05-19 02:52:04.639499: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55a0b000aa00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-05-19 02:52:04.639543: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-05-19 02:52:04.783329: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55a0b0074a20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-19 02:52:04.783399: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A40, Compute Capability 8.6
2023-05-19 02:52:04.784475: I tenso

Epoch 1/400


2023-05-19 02:58:08.166282: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


378/391 [============================>.] - ETA: 0s - loss: 0.4953 - accuracy: 0.7578WARNING:tensorflow:From /global/home/users/mfong/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/resource_variable_ops.py:1813: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


...

391/391 [==============================] - 14s 35ms/step - loss: 0.3643 - accuracy: 0.8380 - val_loss: 0.3784 - val_accuracy: 0.8320
Epoch 67/400
391/391 [==============================] - 12s 30ms/step - loss: 0.3630 - accuracy: 0.8389 - val_loss: 0.3826 - val_accuracy: 0.8279
Epoch 68/400
391/391 [==============================] - 12s 30ms/step - loss: 0.3622 - accuracy: 0.8378 - val_loss: 0.3877 - val_accuracy: 0.8258
Epoch 69/400
391/391 [==============================] - 12s 30ms/step - loss: 0.3627 - accuracy: 0.8389 - val_loss: 0.3837 - val_accuracy: 0.8291
Epoch 70/400
391/391 [==============================] - 12s 30ms/step - loss: 0.3608 - accuracy: 0.8397 - val_loss: 0.3863 - val_accuracy: 0.8257
Epoch 71/400
391/391 [==============================] - 12s 30ms/step - loss: 0.3623 - accuracy: 0.8381 - val_loss: 0.3846 - val_accuracy: 0.8289
Epoch 72/400
391/391 [==============================] - 12s 30ms/step - loss: 0.3604 - accuracy: 0.8398 - val_loss: 0.3866 - val_accuracy

wandb: Adding directory to artifact (/global/home/users/mfong/git/transfer-learning/wandb/run-20230519_025153-ckir1988/files/model-best)... Done. 0.0s


391/391 [==============================] - 14s 35ms/step - loss: 0.3464 - accuracy: 0.8471 - val_loss: 0.3782 - val_accuracy: 0.8333
Epoch 100/400
391/391 [==============================] - 12s 30ms/step - loss: 0.3480 - accuracy: 0.8458 - val_loss: 0.3789 - val_accuracy: 0.8332
Epoch 101/400
391/391 [==============================] - 12s 30ms/step - loss: 0.3454 - accuracy: 0.8463 - val_loss: 0.3870 - val_accuracy: 0.8273
Epoch 102/400
391/391 [==============================] - 12s 31ms/step - loss: 0.3457 - accuracy: 0.8474 - val_loss: 0.3859 - val_accuracy: 0.8317
Epoch 103/400
391/391 [==============================] - 12s 31ms/step - loss: 0.3469 - accuracy: 0.8469 - val_loss: 0.3872 - val_accuracy: 0.8292
Epoch 104/400
391/391 [==============================] - 12s 31ms/step - loss: 0.3456 - accuracy: 0.8471 - val_loss: 0.3822 - val_accuracy: 0.8312
Epoch 105/400
391/391 [==============================] - 12s 30ms/step - loss: 0.3445 - accuracy: 0.8476 - val_loss: 0.3810 - val_ac

wandb: Adding directory to artifact (/global/home/users/mfong/git/transfer-learning/wandb/run-20230519_025153-ckir1988/files/model-best)... Done. 0.0s


391/391 [==============================] - 14s 35ms/step - loss: 0.3452 - accuracy: 0.8475 - val_loss: 0.3773 - val_accuracy: 0.8320
Epoch 107/400
391/391 [==============================] - 12s 30ms/step - loss: 0.3457 - accuracy: 0.8465 - val_loss: 0.3799 - val_accuracy: 0.8320
Epoch 108/400
391/391 [==============================] - 12s 31ms/step - loss: 0.3433 - accuracy: 0.8490 - val_loss: 0.3810 - val_accuracy: 0.8305
Epoch 109/400
391/391 [==============================] - 12s 30ms/step - loss: 0.3420 - accuracy: 0.8491 - val_loss: 0.3810 - val_accuracy: 0.8344
Epoch 110/400
391/391 [==============================] - 12s 30ms/step - loss: 0.3450 - accuracy: 0.8463 - val_loss: 0.3793 - val_accuracy: 0.8323
Epoch 111/400
391/391 [==============================] - 12s 32ms/step - loss: 0.3421 - accuracy: 0.8488 - val_loss: 0.3911 - val_accuracy: 0.8292
Epoch 112/400
391/391 [==============================] - 12s 31ms/step - loss: 0.3429 - accuracy: 0.8488 - val_loss: 0.3828 - val_ac